## Question 4
---------
Faites une analyse détaillée et exhaustive des patrons existants dans le jeu de données adulte.
Il est possible que le regroupement de valeurs pour certaines caractéristiques donne des résultat intéressant.
Vous devez aussi, s’il y a lieux, discuter de l’aspect éthique concernant l’utilisation des patrons obtenus.

In [64]:
%matplotlib inline
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [65]:
dataSet= []
idx = 0

with open('adult.csv') as csv_file:
    reader = csv.reader(csv_file, delimiter=' ')
    for i, row in enumerate(reader):            
        dataSet.append(row[:-1])        

In [66]:
# cette fonction prends une liste C1, la trier, mapper chaque élément de la liste sur frozenset () 
#et renvoyez cette liste de frozensets.
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))#utilise un ensemble figé pour pouvoir l'utiliser comme clé dans un dictionnaire

In [67]:
C1 = createC1(dataSet)

In [68]:
D = list(map(set,dataSet))

In [69]:
D

[{'Age,',
  'capital-gain,',
  'capital-loss,',
  'education,',
  'education-num,',
  'fnlwgt,',
  'hours-per-week,',
  'marital-status,',
  'native-country,',
  'occupation,',
  'race,',
  'relationship,',
  'sex,',
  'workclass,'},
 {'0,',
  '13,',
  '2174,',
  '39,',
  '40,',
  '77516,',
  'Adm-clerical,',
  'Bachelors,',
  'Male,',
  'Never-married,',
  'Not-in-family,',
  'State-gov,',
  'United-States,',
  'White,'},
 {'0,',
  '13,',
  '50,',
  '83311,',
  'Bachelors,',
  'Exec-managerial,',
  'Husband,',
  'Male,',
  'Married-civ-spouse,',
  'Self-emp-not-inc,',
  'United-States,',
  'White,'},
 {'0,',
  '215646,',
  '38,',
  '40,',
  '9,',
  'Divorced,',
  'HS-grad,',
  'Handlers-cleaners,',
  'Male,',
  'Not-in-family,',
  'Private,',
  'United-States,',
  'White,'},
 {'0,',
  '11th,',
  '234721,',
  '40,',
  '53,',
  '7,',
  'Black,',
  'Handlers-cleaners,',
  'Husband,',
  'Male,',
  'Married-civ-spouse,',
  'Private,',
  'United-States,'},
 {'0,',
  '13,',
  '28,',
  '33840

In [70]:
#Cette fonction prend trois arguments: un ensemble de données, Ck, une liste d'ensembles candidats et un support min, 
#qui est le support minimum qui vous intéresse. Cette fonction est utilisée pour générer L1 à partir de C1. 
#De plus, cette fonction retourne un dictionnaire avec des valeurs de support.
#C1 est un groupe d’éléments candidat de taille 1. Dans l'algorithme Apriori, nous créons C1, puis nous analysons 
#l'ensemble de données pour voir si ces ensembles d'éléments répondent à nos exigences de support minimales. 
#Les ensembles d'éléments qui répondent à nos exigences minimales deviennent L1. 
#L1 est ensuite combiné pour devenir C2 et C2 sera filtré pour devenir L2.
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData


In [71]:
#La fonction aprioriGen () prendra une liste d'itemsets fréquents, Lk, et la taille des itemsets,
#k, pour produire Ck. Par exemple, les itemsets {0}, {1}, {2} et ainsi de suite seront générés {0,1} {0,2} et {1,2}.
def aprioriGen(Lk, k): #Creer Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #si k-2 premier elements sont egaux
                retList.append(Lk[i] | Lk[j]) #ajouter ensemble union
    return retList

In [72]:
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#balayer DB pour obtenir Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData #L contient des listes de jeux d'éléments fréquents qui ont rencontré un support minimum de 0.5. 
                          #La variable suppData est un dictionnaire avec les valeurs de support de nos itemsets

In [73]:
#La fonction generateRules () prend trois entrées: une liste d'ensembles d'éléments fréquents, 
#un dictionnaire de données de support pour ces ensembles d'éléments et un seuil de confiance minimal. 
#Il va générer une liste de règles avec des valeurs de confiance que nous pourrons trier plus tard.
def generateRules(L, supportData, minConf=0.7):  #supportData est un dictionnaire qui vient de la fonction scanD 
    bigRuleList = []
    for i in range(1, len(L)):#n'obtenez que les ensembles contenant deux objets ou plus
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList  

In [74]:
L,suppData = apriori(dataSet)

In [75]:
rules= generateRules(L,suppData, minConf=0.7)

frozenset({'Private,'}) --> frozenset({'0,'}) conf: 1.0
frozenset({'Private,'}) --> frozenset({'United-States,'}) conf: 0.8871607331688403
frozenset({'Private,'}) --> frozenset({'White,'}) conf: 0.8549524145223828
frozenset({'Male,'}) --> frozenset({'0,'}) conf: 1.0
frozenset({'United-States,'}) --> frozenset({'0,'}) conf: 1.0
frozenset({'0,'}) --> frozenset({'United-States,'}) conf: 0.8958570068486841
frozenset({'Male,'}) --> frozenset({'United-States,'}) conf: 0.8943552088113815
frozenset({'White,'}) --> frozenset({'0,'}) conf: 1.0
frozenset({'0,'}) --> frozenset({'White,'}) conf: 0.8542735173981143
frozenset({'Male,'}) --> frozenset({'White,'}) conf: 0.8799449288664525
frozenset({'White,'}) --> frozenset({'United-States,'}) conf: 0.9210885821110153
frozenset({'United-States,'}) --> frozenset({'White,'}) conf: 0.8783339046966061
frozenset({'Private,'}) --> frozenset({'United-States,', 'White,'}) conf: 0.7811068029608742
frozenset({'Private,'}) --> frozenset({'0,', 'White,'}) conf: 0.